<a href="https://colab.research.google.com/github/IsakLarsson/Deep-Learning-labs/blob/main/Word2vec_Isak_%C3%85man_Larsson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Isak Åman Larsson

##Importera nyttiga saker till miljön

In [114]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
from keras.utils import to_categorical
from keras import models
from keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

Använder mig även här av IMDB datasetet, när man importerar det via tensorflows egna load_data så kommer det färdig encodat. Alltså måste jag konvertera tillbaks det för att använda word2vec

In [115]:
from keras.datasets import imdb

# ta bara top X orden, denna kan lekas med och tas in i load_data, annars tas allt med
NUM_WORDS=1000
INDEX_FROM=3   # från vilket index orden börjar indexeras

train,test = imdb.load_data(
    #num_words=NUM_WORDS,
    index_from=INDEX_FROM
    )
train_x,train_y = train
test_x,test_y = test

word_to_id = imdb.get_word_index()
word_to_id = {k:(v+INDEX_FROM) for k,v in word_to_id.items()}
word_to_id["<PAD>"] = 0
word_to_id["<START>"] = 1
word_to_id["<OOV>"] = 2
word_to_id["<UNUSED>"] = 3

id_to_word = {value:key for key,value in word_to_id.items()}

#konvertera från IDn till ord och lägg till i en lista
review_lines= list()
for review in train_x:
  word_list = []
  for wordID in review:
    word_list.append(id_to_word[wordID])

  review_lines.append(word_list)

# Printa en godtycklig review
print(review_lines[2])

['<START>', 'this', 'has', 'to', 'be', 'one', 'of', 'the', 'worst', 'films', 'of', 'the', '1990s', 'when', 'my', 'friends', 'i', 'were', 'watching', 'this', 'film', 'being', 'the', 'target', 'audience', 'it', 'was', 'aimed', 'at', 'we', 'just', 'sat', 'watched', 'the', 'first', 'half', 'an', 'hour', 'with', 'our', 'jaws', 'touching', 'the', 'floor', 'at', 'how', 'bad', 'it', 'really', 'was', 'the', 'rest', 'of', 'the', 'time', 'everyone', 'else', 'in', 'the', 'theatre', 'just', 'started', 'talking', 'to', 'each', 'other', 'leaving', 'or', 'generally', 'crying', 'into', 'their', 'popcorn', 'that', 'they', 'actually', 'paid', 'money', 'they', 'had', 'earnt', 'working', 'to', 'watch', 'this', 'feeble', 'excuse', 'for', 'a', 'film', 'it', 'must', 'have', 'looked', 'like', 'a', 'great', 'idea', 'on', 'paper', 'but', 'on', 'film', 'it', 'looks', 'like', 'no', 'one', 'in', 'the', 'film', 'has', 'a', 'clue', 'what', 'is', 'going', 'on', 'crap', 'acting', 'crap', 'costumes', 'i', "can't", 'get'

Denna review va inte speciellt positiv...

##Importera Word2Vec modellen och kör reviewsen genom den

In [116]:
import gensim

EMBEDDING_DIM = 100
WINDOW_SIZE = 5

model = gensim.models.Word2Vec(sentences=review_lines, size=EMBEDDING_DIM, window=WINDOW_SIZE, workers=4, min_count=1 )
words = list(model.wv.vocab)
print("Vocabulary size (mängden ord): ", len(words))

Vocabulary size (mängden ord):  88585


###Testa liknande ord

Vi kan nu köra några exempel på liknande ord, här tar vi ut top 3 liknande orden för några godtyckliga ord

In [117]:
print("Amazing resembles: ", model.wv.most_similar('amazing')[:3])
print("Horrible resembles: ", model.wv.most_similar('horrible')[:3])
print("Movie resembles: ", model.wv.most_similar('movie')[:3])
print("Dog resembles: ", model.wv.most_similar('dog')[:3])
print("Superhero resembles: ", model.wv.most_similar('superhero')[:3])

print('\n\n')

Amazing resembles:  [('incredible', 0.8454157114028931), ('awesome', 0.8207459449768066), ('excellent', 0.7816787958145142)]
Horrible resembles:  [('terrible', 0.9374001622200012), ('awful', 0.8611143827438354), ('laughable', 0.7919660806655884)]
Movie resembles:  [('film', 0.9302455186843872), ('flick', 0.7232173085212708), ('show', 0.6845262050628662)]
Dog resembles:  [('cat', 0.7753405570983887), ('rat', 0.7121865749359131), ('bottle', 0.711240291595459)]
Superhero resembles:  [('gangster', 0.797494113445282), ('vampire', 0.7227295637130737), ('blaxploitation', 0.7127607464790344)]





/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Vet inte vad jag tycker om att superhero är likt wannabe, demented och gangster...

Vi testar lite "ord matematik"

In [118]:
model.wv.most_similar_cosmul(positive=['woman','king'], negative=['man'])

[('jane', 0.9065161943435669),
 ('eyre', 0.8941454291343689),
 ('queen', 0.8744903802871704),
 ('austen', 0.8731036186218262),
 ('nancy', 0.8545281291007996),
 ('poppins', 0.8515877723693848),
 ('mary', 0.8512917757034302),
 ('alice', 0.84878009557724),
 ('bride', 0.8464763164520264),
 ('elizabeth', 0.8437730073928833)]

In [119]:
model.wv.most_similar_cosmul(positive=['movie','film'], negative=['script'])

[('flick', 1.1458104848861694),
 ('documentary', 1.0899816751480103),
 ('cinema', 1.0817954540252686),
 ('picture', 1.045541763305664),
 ('movies', 1.035487413406372),
 ('show', 1.0194255113601685),
 ('program', 1.0079246759414673),
 ('films', 1.0069624185562134),
 ('episode', 0.986014723777771),
 ('series', 0.9820832014083862)]

In [120]:
model.wv.most_similar_cosmul(positive=['actor','woman'], negative=['man'])

[('actress', 1.0699996948242188),
 ('role', 0.8864448070526123),
 ('ms', 0.8801533579826355),
 ('accent', 0.874535858631134),
 ('winslett', 0.8656671643257141),
 ('performance', 0.8636323809623718),
 ('singer', 0.8610719442367554),
 ('performer', 0.8485500812530518),
 ('comedienne', 0.8470349907875061),
 ('dancer', 0.84434574842453)]

Väldigt intressant att se hur modellen har placerat ord nära varandra. I vissa fall känns det väldigt självklart som att "amazing" liknar "incredible" men ibland blir det konstigt som att superhero liknar gangster. Samma sak uppstår vid beräkning av vektorerna, woman + king - man = jane. queen hamnar längre ner på listan men det finns i alla fall med. Dessa associationer beror mycket på hur stort window man ger modellen, kontexten blir helt annorlunda. Just för detta dataset är det även såklart smått begränsat till film termer. I alla fall om man vill få de bästa resultaten.